# Intervalo binomial

Calculamos el intervalo de confianza para el parámetro *probabilidad de éxito* $p$ de una variable binomial, $k \sim B(n,p)$, dónde $n$ es el número de pruebas de Bernoulli.

El número de pruebas de Bernoulli está fijado por experimento:

In [ ]:
ntrials = 100

El número de pruebas exitosas de nuestro experimento fue:

In [ ]:
k_obs = 66

El estimador de máxima verosimilitud del parámetro $\hat{p}$ es:

In [ ]:
proba_est = k_obs / ntrials
proba_est

## Superficie de nivel

Calculamos el intervalo de confianza con la superficie de nivel de la función de costo. Para esto usamos de la librería [`likefit`](https://pypi.org/project/likefit/), la función de costo binomial, $ J(p) = 2 \, n \left[  \hat{p} \, \ln\left( \frac{\hat{p}}{p} \right) +  (1-\hat{p}) \, \ln\left( \frac{1-\hat{p}}{1-p} \right)  \right]$:

Install the library likefit if not already installed

In [ ]:
pip install likefit

In [ ]:
import likefit
def cost(x):
    return likefit.binomial_cost(x, k_obs, ntrials) 

Debemos encontrar los valores del parámetro $p$ a los que la función de costo es igual $m^2$ para encontrar el intervalo $m$σ. Realizamos este procedimiento buscando la raíz de la función de costo menos $m^2$: 

In [ ]:
msigma = 1
def delta_cost(x):
    return cost(x) - msigma**2

In [ ]:
from scipy.optimize import brentq
proba_down = brentq(delta_cost, a=0.01, b=proba_est)  
proba_up = brentq(delta_cost, a=proba_est, b=0.99) 
print(f"Confidence interval: [{proba_down:.3f},{proba_up:.3f}]")

Reportamos un intervalo asimétrico:

In [ ]:
error_down = proba_est - proba_down
error_up =  proba_up - proba_est
print(f"p = {proba_est} +{error_up:.3f} -{error_down:.3f}")

## Errores parabólicos 

De esta forma calculamos númericamente el intervalo de confianza con la función de verosimilitud. También podríamos calcular analíticamente los errores parabólicos $\delta p = m \, \left( \frac{1}{2} \, \frac{\partial^2 J(p)}{ \partial p^2 }\Big|_{\hat{p}} \right)^{-1/2} = m \, \sqrt{ \frac{\hat{p} \, (1-\hat{p})}{n}   }$:

In [ ]:
import math
parabolic_error = msigma * math.sqrt(proba_est * (1 - proba_est) / ntrials )
print(f"p = {proba_est} ±  {parabolic_error:.3f}")

Los errores parabólicos son iguales a los que obtendríamos de aproximar la distribución de $\hat{p}$ por una normal. Veamos primero que la media del estimador es igual al valor del parámetro, $\mathrm{E}(\hat{p}) = p$; esto es, no tiene sesgo. A su vez, la varianza es $\mathrm{Var}(\hat{p}) = (1-p) p / n$. Como no conocemos el parámetro $p$, para estimar la varianza usamos el valor del estimador en su lugar, $\mathrm{Var}(\hat{p}) = (1-\hat{p}) \hat{p} / n$. El error de $p$ para $\hat{p}$ normal es la desviación estándar de $\hat{p}$, que justamente coincide con la definición de $\delta p$ que dimos más arriba. 

## Plot del intervalo

Definimos un rango de algunos pocos sigmas alrededor del estimador de máxima verosimilitud para graficar la función de costo:

In [ ]:
import numpy as np
proba_min = proba_est - (msigma+1) * parabolic_error
proba_max = proba_est + (msigma+1) * parabolic_error
proba = np.linspace(proba_min, proba_max, num=100) 

In [ ]:
cost = likefit.binomial_cost(proba, k_obs, ntrials)

Ploteamos la función de costo

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.set_xlabel("Bernoulli probability")
ax.set_ylabel("Cost")
ax.plot(proba, cost)
ax.axhline(0, ls='--', color='tab:gray')
ax.axhline(msigma**2, ls='--', color='tab:orange', label="1σ")
ax.vlines(x=proba_down, ymin=0, ymax=msigma**2)
ax.text(proba_down, 0.5, "$p_1$ ", horizontalalignment='right')
ax.vlines(x=proba_up, ymin=0, ymax=msigma**2)
ax.text(proba_up, 0.5, " $p_2$", horizontalalignment='left')
fig.tight_layout()
# fig.savefig("intervalo_binomial.svg")